Data was published at https://github.com/duyvuleo/VNTC

# Introduction

In this tutorial, we will implement some algorithms to apply in text summarization problem.

## What is Text Summarization?

Text summarization is the problem of creating a short, accurate, and fluent summary of a longer text document.

Automatic text summarization methods are greatly needed to address the ever-growing amount of text data available online to both better help discover relevant information and to consume relevant information faster.

## What will we do in this tutorial?

In this tutorial, we will solve Text Summarization for Vietnamese newspapers, using some algorithms belows:
1. Extractive Text Summarization
    - Doc2Vec
    - Text Rank
2. Abstractive Text Summarization
    - Google textsum


We just implement "**Single document summarization**" problem in this tutorial, another problem called "**Multi-document summarization**" will be dicussed in another time.

# Extractive Text Summarization

## Doc2Vec

example: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb

### Basic idea
The idea of using Doc2Vec algorithm for text summarization problem is described as follows:
1. In all documents, we will extract sentences separately.
2. Each sentence will be represented by a vector, via doc2vec model
3. Use KMean algorithm to find out most featured sentences.

In [8]:
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
from pyvi import ViTokenizer, ViPosTagger
from tqdm import tqdm
import numpy as np
import gensim
import numpy as np

In [10]:
import os 
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
dir_path = os.path.join(dir_path, 'Data')

sentences = []

In [11]:
import pickle

def get_data(folder):
    sentences = []
    for path in os.listdir(folder):
        file_path = os.path.join(folder, path)
        with open(file_path, 'r', encoding="utf-16") as f:

            lines = f.readlines()

            for line in lines:
                sens = line.split('.')
                for sen in sens:
                    if len(sen) > 10:
                        sen = gensim.utils.simple_preprocess(sen)
                        sen = ' '.join(sen)
                        sen = ViTokenizer.tokenize(sen)
                        sentences.append(sen)

    return sentences

In [6]:
# sens = test_doc.split('.')
# for sen in sens:
#     if len(sen) > 10:
#         sen = gensim.utils.simple_preprocess(sen)
#         sen = ' '.join(sen)
#         sen = ViTokenizer.tokenize(sen)
#         sentences.append(sen)

In [7]:
# sentences

You can use multiprocessing here, but we will not use it for easy in understanding code.

In [10]:
# from multiprocessing import Pool
# sentences = []
# train_paths = [os.path.join(dir_path, 'VNTC-master/Data/10Topics/Ver1.1/Train_Full'), 
#                os.path.join(dir_path, 'VNTC-master/Data/10Topics/Ver1.1/Test_Full'),
#                os.path.join(dir_path, 'VNTC-master/Data/27Topics/Ver1.1/new train'),
#                os.path.join(dir_path, 'VNTC-master/Data/27Topics/Ver1.1/new test')]

# dirs = []
# for path in train_paths:
#     for p in os.listdir(path):
#         dirs.append(os.path.join(path, p))

# for d in tqdm(dirs):
#     sens = get_data(d)
#     sentences = sentences + sens

# # with Pool(8) as pool:
# #     pool.map(get_data, tqdm(dirs))



In [12]:
# pickle.dump(sentences, open('./sentences.pkl', 'wb'))
sentences = pickle.load(open('./sentences.pkl', 'rb'))

In [13]:
sentences[0]

'ông đồ cuối_cùng trên đảo'

In [14]:
def get_corpus(sentences):
    corpus = []
    
    for i in tqdm(range(len(sentences))):
        sen = sentences[i]
        
        words = sen.split(' ')
        tagged_document = gensim.models.doc2vec.TaggedDocument(words, [i])
        
        corpus.append(tagged_document)
        
    return corpus

In [15]:
train_corpus = get_corpus(sentences)

100%|██████████| 2385532/2385532 [00:34<00:00, 69769.71it/s]


In [11]:
from sklearn.utils import shuffle

train_corpus = shuffle(train_corpus)

#### Build Doc2Vec model

In [11]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=40)
model.build_vocab(train_corpus)

In [13]:
# max_epochs = 40

# for epoch in tqdm(range(max_epochs)):
#     print('iteration {0}'.format(epoch))
model.train(train_corpus[:50000],
                total_examples=model.corpus_count,
                epochs=model.iter)
    
#     # decrease the learning rate
#     model.alpha -= 0.0002
#     # fix the learning rate, no decay
#     model.min_alpha = model.alpha

# %time model.train(train_corpus[:50000], total_examples=model.corpus_count, epochs=model.epochs)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  import sys


In [15]:
model.save('model/model')

In [12]:
model = gensim.models.doc2vec.Doc2Vec.load('model/model')

In [21]:
model.infer_vector(train_corpus[100000].words)

array([-0.04828287,  0.25527653,  1.1613333 , -0.43151897, -0.9858117 ,
        0.10932952,  0.20315444, -0.48530903,  0.24952224, -0.11833256,
       -0.0337567 , -0.3887124 , -0.39426357,  0.4454976 ,  0.64964545,
       -0.5074249 ,  0.2037328 ,  0.32153234, -0.62261915,  0.8188216 ,
        0.5820815 , -0.09879603, -0.44826344,  0.1201525 ,  0.236654  ,
        0.13032307, -0.46023956,  0.19788027, -0.34569028, -0.21599784,
        0.42319658, -0.106575  , -0.24495657, -0.00839793, -0.11475623,
       -0.5559897 , -0.12046688,  0.18673038, -0.16149993, -0.02872676,
        0.42999822,  0.46070522,  0.50624824, -0.15866163, -0.11092521,
        0.30938515,  0.23203233,  0.11736044, -0.7434822 , -0.78674805,
        0.27668393,  0.25058967, -0.15513541,  0.05721006, -0.62895125,
       -0.3618494 ,  0.48457113, -0.16074707,  0.32852057, -0.63208133,
       -0.45503548, -0.373764  ,  0.6417061 , -0.15453526,  0.828889  ,
        0.4040729 , -0.13313939,  0.20088702, -0.36382645,  0.31

#### Test with new document

In [1]:
test_doc = '''Trong trận bán kết lượt về AFF Cup 2018 diễn ra trên sân vận động Mỹ Đình tối 6/12, đội tuyển Việt Nam đã vượt qua đội tuyển Philippines với tỉ số 2-1. Qua đó, nâng tổng tỉ số sau hai lượt trận bán kết là 4-2.

Đội tuyển Việt Nam đã xuất sắc giành quyền vào chơi trận chung kết AFF Cup sau tròn 10 năm chờ đợi. Đối thủ của chúng ta là đội tuyển Malaysia.

Hai cầu thủ ghi bàn thắng trên sân Mỹ Đình tối qua là Quang Hải và Công Phượng. Đáng chú ý, bàn thắng của Công Phượng được ghi chỉ sau vài phút anh được HLV Park Hang Seo tung vào sân thay người ở những phút cuối cùng của trận đấu.

Bàn thắng của Công Phượng không khỏi khiến nhiều người nhớ đến pha bỏ lỡ “không tưởng” của cầu thủ này ở trận bán kết lượt đi trên sân của đội tuyển Philippines hôm 2/12.

Trong trận đấu ấy, Công Phượng cũng được HLV trưởng người Hàn Quốc tung vào sân ở những phút cuối trận đấu. Anh thực hiện một pha đi bóng qua hàng loạt cầu thủ hậu vệ Philippines. Thế nhưng, khi đối mặt với khung thành rộng lớn, anh lại sút bóng chệch cột dọc.

Sau tình huống bỏ lỡ ấy, cộng đồng mạng Việt Nam thi nhau chế ảnh Công Phượng. Họ cho rằng, Công Phượng không chỉ lừa qua hàng loạt hậu vệ Philippines mà còn lừa luôn cả hàng triệu fan hâm mộ đội nhà.

Thắng bán kết AFF Cup 2018, Công Phượng hết &#34;lừa&#34; fan, Văn Toàn hứa hẹn trở lại - 2

Công Phượng đã không còn lừa người hâm mộ khi ghi bàn trong trận bán kết lượt về AFF Cup 2018.

Chính vì vậy, trước trận đấu bán kết lượt về hôm qua, Công Phượng đã đăng tải một tấm hình lên mạng xã hội Facebook với tựa đề: “Ngày mai rồi đấy”.

Dòng trạng thái ấy thể hiện quyết tâm của tiền đạo xứ Nghệ. Anh mong chờ được ra sân trong trận bán kết lượt về với Philippines để khẳng định mình và lấy lại niềm tin nơi người hâm mộ. Và cuối cùng, Công Phượng cũng đã làm được điều mình mong muốn.

Thắng bán kết AFF Cup 2018, Công Phượng hết &#34;lừa&#34; fan, Văn Toàn hứa hẹn trở lại - 3

Status trước hôm bán kết thể hiện sự quyết tâm của Công Phương.

Ngay sau trận bán kết lượt về kết thúc, Công Phượng lại tiếp tục đăng một status: “Lần này không lừa cả nhà nữa nhé. Thắng rồi bà con ơi”. Với bàn thắng ghi được ở những phút cuối trận đấu, Công Phượng đã giúp đội tuyển Việt Nam chắc chắn vào chơi trận chung kết AFF Cup 2018.

Cũng sau trận đấu bán kết lượt về khi đội tuyển Việt Nam vượt qua đội tuyển Philippines, cầu thủ Văn Toàn đã chia sẻ trạng thái: “Trở lại thôi”. Dòng trạng thái này của Văn Toàn như một thông điệp gửi tới người hâm mộ rằng, anh đã bình phục chấn thương và sẵn sàng trở lại ở trận chung kết.

Thắng bán kết AFF Cup 2018, Công Phượng hết &#34;lừa&#34; fan, Văn Toàn hứa hẹn trở lại - 4

Văn Toàn đăng status mang thông điệp đã bình phục chấn thương và sẵn sàng trở lại.

Thắng bán kết AFF Cup 2018, Công Phượng hết &#34;lừa&#34; fan, Văn Toàn hứa hẹn trở lại - 5

Người hâm mộ động viên tinh thần khi biết Văn Toàn sắp trở lại.

Trước đó, Văn Toàn đã bị chấn thương sụn chêm ở đầu gối sau một pha va chạm với đồng đội Văn Quyết trong buổi tập trước trận đấu với đội tuyển Campuchia ở vòng bảng AFF Cup 2018.

Rất may, chấn thương của Văn Toàn không quá nặng và không phải phẫu thuật nên bình phục nhanh chóng. Ban đầu, các bác sĩ của đội tuyển Việt Nam dự đoán Văn Toàn có thể trở lại ở trận bán kết lượt về. Tuy nhiên, chấn thương chưa bình phục hẳn nên Văn Toàn phải đợi đến chung kết để có cơ hội được ra sân.

Những cầu thủ khác như Nguyễn Quang Hải, Phan Văn Đức, Phạm Đức Huy cũng có những chia sẻ lên Facebook cá nhân sau trận đấu. Các cầu thủ thầm cảm ơn những người thân, người hâm mộ đã luôn bên họ và chứng kiến họ trưởng thành.
'''

In [9]:
test_doc

'Trong trận bán kết lượt về AFF Cup 2018 diễn ra trên sân vận động Mỹ Đình tối 6/12, đội tuyển Việt Nam đã vượt qua đội tuyển Philippines với tỉ số 2-1. Qua đó, nâng tổng tỉ số sau hai lượt trận bán kết là 4-2.\n\nĐội tuyển Việt Nam đã xuất sắc giành quyền vào chơi trận chung kết AFF Cup sau tròn 10 năm chờ đợi. Đối thủ của chúng ta là đội tuyển Malaysia.\n\nHai cầu thủ ghi bàn thắng trên sân Mỹ Đình tối qua là Quang Hải và Công Phượng. Đáng chú ý, bàn thắng của Công Phượng được ghi chỉ sau vài phút anh được HLV Park Hang Seo tung vào sân thay người ở những phút cuối cùng của trận đấu.\n\nBàn thắng của Công Phượng không khỏi khiến nhiều người nhớ đến pha bỏ lỡ “không tưởng” của cầu thủ này ở trận bán kết lượt đi trên sân của đội tuyển Philippines hôm 2/12.\n\nTrong trận đấu ấy, Công Phượng cũng được HLV trưởng người Hàn Quốc tung vào sân ở những phút cuối trận đấu. Anh thực hiện một pha đi bóng qua hàng loạt cầu thủ hậu vệ Philippines. Thế nhưng, khi đối mặt với khung thành rộng lớn, a

In [44]:
def get_list_sentence_vectors_from_document(doc, model):
    vectors = []
    sens = doc.split('.')
    for sen in sens:
        if len(sen) > 10:
            sen = gensim.utils.simple_preprocess(sen)
            sen = ' '.join(sen)
            sen = ViTokenizer.tokenize(sen)
            sen = sen.split(' ')
            vec = model.infer_vector(sen)
            
            vectors.append(vec)
    
    return np.array(vectors), sens

In [45]:
sen_vectors, sens = get_list_sentence_vectors_from_document(test_doc, model=model)

NameError: name 'model' is not defined

In [46]:
sen_vectors.shape

NameError: name 'sen_vectors' is not defined

In [59]:
X = sen_vectors

In [62]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
n_clusters = 2

gm = GaussianMixture(2)
gm.fit(X)
# kmeans = KMeans(n_clusters=n_clusters)
# kmeans = kmeans.fit(X)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=2, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [64]:
gm.weights_

array([0.07272727, 0.92727273])

In [63]:
from sklearn.metrics import pairwise_distances_argmin_min

avg = []
for j in range(n_clusters):
    idx = np.where(kmeans.labels_ == j)[0]
    avg.append(np.mean(idx))
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)
ordering = sorted(range(n_clusters), key=lambda k: avg[k])
summary = [sens[closest[idx]] for idx in ordering]

for sen in summary:
    print(sen)



Thắng bán kết AFF Cup 2018, Công Phượng hết &#34;lừa&#34; fan, Văn Toàn hứa hẹn trở lại - 5

Người hâm mộ động viên tinh thần khi biết Văn Toàn sắp trở lại


Hai cầu thủ ghi bàn thắng trên sân Mỹ Đình tối qua là Quang Hải và Công Phượng


## Text Rank

In [2]:
def build_index(links):
    website_list = links.keys()
    return {website: index for index, website in enumerate(website_list)}
 
    

In [3]:
import numpy as np
 
def build_transition_matrix(links, index):
    total_links = 0
    A = np.zeros((len(index), len(index)))
    for webpage in links:
        # dangling page
        if not links[webpage]:
            # Assign equal probabilities to transition to all the other pages
            A[index[webpage]] = np.ones(len(index)) / len(index)
        else:
            for dest_webpage in links[webpage]:
                total_links += 1
                A[index[webpage]][index[dest_webpage]] = 1.0 / len(links[webpage])
 
    return A

In [4]:
def pagerank(A, eps=0.0001, d=0.85):
    P = np.ones(len(A)) / len(A)
    while True:
        new_P = np.ones(len(A)) * (1 - d) / len(A) + d * A.T.dot(P)
        delta = abs(new_P - P).sum()
        if delta <= eps:
            return new_P
        P = new_P

In [5]:
from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance
 
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [6]:
def build_similarity_matrix(sentences, stopwords=None):
    # Create an empty similarity matrix
    S = np.zeros((len(sentences), len(sentences)))
 
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
 
            S[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
 
    # normalize the matrix row-wise
    for idx in range(len(S)):
        S[idx] /= S[idx].sum()
 
    return S

In [7]:
def get_list_of_sentences(doc):
    sentences = []
    sens = doc.split('.')
    for sen in sens:
        if len(sen) > 10:
            sen = gensim.utils.simple_preprocess(sen)
            sen = ' '.join(sen)
            sen = ViTokenizer.tokenize(sen)
            sen = sen.split(' ')
#             print(sen)
            sentences.append(sen)
    
    return sentences

In [8]:
sentences = get_list_of_sentences(test_doc)

NameError: name 'gensim' is not defined

In [ ]:
len(sentences)

In [55]:
stop_words = []
S = build_similarity_matrix(sentences, stop_words)    
# print(S)

In [56]:
from operator import itemgetter 

In [57]:
def textrank(sentences, top_n=5, stopwords=None):
    S = build_similarity_matrix(sentences, stop_words) 
    sentence_ranks = pagerank(S)
 
    # Sort the sentence ranks
    ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    selected_sentences = sorted(ranked_sentence_indexes[:top_n])
    summary = itemgetter(*selected_sentences)(sentences)
    return summary
 
for idx, sentence in enumerate(textrank(sentences, top_n=2, stopwords=[])):
    print("%s. %s" % ((idx + 1), ' '.join(sentence)))

1. bàn thắng của công phượng không khỏi khiến nhiều người nhớ đến pha bỏ lỡ không_tưởng của cầu_thủ này trận bán_kết lượt đi trên sân của đội_tuyển philippines hôm
2. thắng bán_kết aff cup công phượng hết lừa fan văn_toàn hứa_hẹn trở_lại công phượng đã không còn lừa người hâm_mộ khi ghi_bàn trong trận bán_kết lượt về aff cup
